# Step 1. Data extraction

In [24]:
import pandas as pd

df = pd.read_csv("db_table.csv")

# Step 2. Data transformation

Creating "bitCode" column:
* 1 - if source equals S1, 
* 0 otherwise.

In [25]:
def categorize_source(source: str):
    if source == "S1":
        return 1
    return 0

df["bitCode"] = df["source"].apply(categorize_source)

Creating "siCode" column:
* A - if bitCode equals 1 and code3 is AP or AH, 
* B - if bitCode equals 1 and code3 is PRD, 
* BpA - if bitCode equals 1 and code3 is YLD, 
* H - if bitCode equals 0 and code3 is AP or AH, 
* T - if bitCode equals 0 and code3 is PRD, 
* TpH - if bitCode equals 0 and code3 is YLD, 
* None otherwise. 

In [26]:
si_code_mapping = {
    (1, "AP"): "A",
    (1, "AH"): "A",
    (1, "PRD"): "BpA",
    (1, "YLD"): "B",
    (0, "AP"): "H",
    (0, "AH"): "H",
    (0, "PRD"): "TpH",
    (0, "YLD"): "T"
}

df["siCode"] = (
    df[["bitCode", "code3"]]
    .apply(
        lambda row: si_code_mapping.get((row["bitCode"], row["code3"]), None), axis=1
    )
)

# Step 3. Data loading
Preparation for data transfer to frontend

In [50]:
def process_request(input_request: dict):
    code1 = input_request["code1"]
    code2 = input_request["code2"]

    filtered_df = df[
        (df["code1"].str.startswith(code1)) &
        (df["code2"].str.split("/").str[0] == code2)
    ]
    
    all_sources = filtered_df["source"].unique().tolist()
    response_all_sources = {(code1, code2): all_sources}
    
    selected_columns = ["updateDate", "code1", "code2", "code3", "value", "bitCode", "siCode"]
    
    response_records_by_sources = {}
    for source in all_sources:
        filtered_data = filtered_df[
            filtered_df["source"] == source
        ][selected_columns]
        
        response_records_by_sources[
            (code1, code2, source)
        ] = filtered_data.to_dict(orient="records")
        
    return response_all_sources, response_records_by_sources

# Example

In [ ]:
input_example = {"code1": "shA", "code2": "W"}

process_request(input_example)